# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770824098385                   -0.53    8.0    161ms
  2   -2.772140551488       -2.88       -1.31    1.0   95.4ms
  3   -2.772170061356       -4.53       -2.57    1.0    100ms
  4   -2.772170668314       -6.22       -3.56    1.0    102ms
  5   -2.772170721585       -7.27       -3.93    2.0    120ms
  6   -2.772170722981       -8.86       -5.47    1.0    107ms
  7   -2.772170723013      -10.49       -5.43    3.0    138ms
  8   -2.772170723015      -11.80       -6.46    1.0    112ms
  9   -2.772170723015      -13.51       -7.29    2.0    122ms
 10   -2.772170723015   +  -13.95       -7.53    1.0    526ms
 11   -2.772170723015      -14.40       -8.51    1.0    107ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770853917426                   -0.53    9.0    197ms
  2   -2.7

1.7735579994070851

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770783431162                   -0.52    9.0    545ms
  2   -2.772060289750       -2.89       -1.32    1.0   97.0ms
  3   -2.772082991169       -4.64       -2.44    1.0    100ms
  4   -2.772083329022       -6.47       -3.11    1.0   99.1ms
  5   -2.772083417315       -7.05       -4.13    2.0    121ms
  6   -2.772083417687       -9.43       -4.59    1.0    104ms
  7   -2.772083417809       -9.91       -5.74    1.0    109ms
  8   -2.772083417811      -11.80       -6.19    2.0    127ms
  9   -2.772083417811      -13.17       -6.63    1.0    114ms
 10   -2.772083417811   +  -14.24       -8.01    1.0    112ms
Solving response problem
[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.76e-03
[ Info: GMRES linsolve in iteration 1